# Assignment 5

You have to get all the votes for all the departments and provinces and districts available in this [link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/T). General a final table with the columns of the department, province, district, political_organization, total_votes.
The code must run and do not fail. If the code fails, it will be graded as zero. Check this.

1. Click on 'Resumen General'.
2. Select 'Presidencial', and then choose 'Peru'.
3. Scrape the data and generate a final table with the following columns: department, province, district, political_organization, total_votes.

Your code must execute successfully; any errors will result in a zero grade. Ensure you test your code thoroughly.

We need to open the libraries and drivers

In [1]:
!pip install selenium
!pip install webdriver-manager
!pip install lxml
!pip install lxmunidecodel

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement lxmunidecodel (from versions: none)
ERROR: No matching distribution found for lxmunidecodel


In [4]:
from selenium import webdriver

In [5]:
from webdriver_manager.chrome import ChromeDriverManager
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import unidecode
from selenium.webdriver.support.ui import Select

We set the driver now

In [6]:
pwd

'C:\\Users\\usuario\\Documents\\GitHub\\MediaLab_Summer_Python\\assignments\\assignment5'

In [44]:
# Case 1 - Download the driver using ChromeDriverManager
service = Service(executable_path="chromedriver-win64/chromedriver.exe")

driver = webdriver.Chrome(service=service)

url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'

driver.get(url)
driver.maximize_window()

Now, we are using xpath to access to "resumen general"

In [45]:
res_gen= driver.find_element (By.XPATH, '/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[1]/div/div/a/div[2]/div')
res_gen.click()

Now, we are using xpath to access to "presidencial"

In [46]:
pres= driver.find_element (By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a')
pres.click()

Now, we are using xpath to access to "Perú"

In [50]:
per= driver.find_element (By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[1]/select/option[2]')
per.click()

It's time to scrape the data looping to get the data from all the departments, provinces and districts

In [61]:
# Store all_tables
all_tables = {}

#1.- departments  
# we need to create a loop for departments
for dept_idx in range( 26 ):
    
    #get all elements from departments
    dept = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    department = dept.options[ dept_idx ]
    
    # Get departmant name
    dept_name = department.text
    
      # We select a different department name
    if dept_name != "--TODOS--" :
        
        # click on department
        department.click()
        
#2.- provinces

        provi = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
        num_provi_options = len( provi.options )
        
# we need to create a loop for provinces
        for provi_idx in range( num_provi_options ):

            # get all elements from provinces
            provi = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            province = provi.options[ provi_idx ]
                
            # Get province name
            provi_name = province.text
            
            if provi_name != "--TODOS--" :
                
                # click on province
                province.click()
                
#3.- districts

                distri = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_distri_options = len( distri.options )
                
# we need to create a loop for districts
                for distri_idx in range( num_distri_options ):
                    
                    #get all elements from districts
                    distri = Select( driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                    district = distri.options[ distri_idx ]
                    
                    # Get district name
                    distri_name = district.text
                   
                  # We select a different district name
                    if distri_name != "-- SELECCIONE --" :
                        
                        # click on district
                        district.click()
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                        ## Get table of presidential votes
                        # Get html at this point
                        table_path = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div" )
                        table_html = table_path.get_attribute( 'innerHTML' )
                        # Read the table using pandas
                        table = pd.read_html( table_html )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( " ", "_") \
                                              .str.lower().str.replace( "%", "share_") \
                                              .apply( lambda x : unidecode.unidecode( x ) ) \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'department' ] = dept_name
                        table_clean[ 'province' ]   = provi_name
                        table_clean[ 'district' ]   = distri_name
                    
                        
                        # store tables
                        all_tables[ ubigeo ] = table_clean

In [62]:
final_table = pd.concat( all_tables.values() ).reset_index( drop = True )

In [63]:
final_table

,organizaciones_politicas,total,share_validos,share_emitidos,department,province,district
0,PARTIDO NACIONALISTA PERUANO,356,9.327%,6.389%,AMAZONAS,BAGUA,ARAMANGO
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,0.341%,0.233%,AMAZONAS,BAGUA,ARAMANGO
2,PARTIDO MORADO,41,1.074%,0.736%,AMAZONAS,BAGUA,ARAMANGO
3,PERU PATRIA SEGURA,4,0.105%,0.072%,AMAZONAS,BAGUA,ARAMANGO
4,VICTORIA NACIONAL,151,3.956%,2.710%,AMAZONAS,BAGUA,ARAMANGO
...,...,...,...,...,...,...,...
41157,ALIANZA PARA EL PROGRESO,292,35.394%,20.434%,UCAYALI,PURUS,PURUS
41158,TOTAL DE VOTOS VÁLIDOS,825,100.000%,57.733%,UCAYALI,PURUS,PURUS
41159,VOTOS EN BLANCO,422,NaN,29.531%,UCAYALI,PURUS,PURUS
41160,VOTOS NULOS,182,NaN,12.736%,UCAYALI,PURUS,PURUS
